# Router Engine

Given a query, the router will pick what to run.  This shows dynamic query understanding.

## Setup

In [1]:
# this is required since Juypyter notebook is not async by default.
import nest_asyncio
nest_asyncio.apply()

## Load data

In [2]:
from llama_index.core import SimpleDirectoryReader

lightning_ai="/teamspace/studios/this_studio/retrieval-augmented-generation/data/transcript.pdf"
github_codespace="./data/transcript.pdf"

docs = lightning_ai
documents = SimpleDirectoryReader(input_files=[docs]).load_data()


## Split document into nodes

In [3]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

## Define LLM and Embedding Model

In [4]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Indexes

- Summary Index 
- Vector Index 

Think of the index as a set of metadata over the data.  Different indexes have different index behavior.  A vector index indexes node through text embeddings.  Querying a vector index in llamaindex will return the most similar nodes. A summary index is also a simple index, and a query to it will return all of the nodes that are currently in the index.

In [5]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Turn the indexes into query engines

Each query engine represents an interface over the data that has been stored in the index.  It combines retrieval with LLM synthesis.  Each engine is specific for a certain kind of question.  This is where the use case of the router becomes significant.  

In [6]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

vector_query_engine = vector_index.as_query_engine()

## Create query tools

The query tools are simply the query engine with additional metadata.  The metadata reflects the kind of information the tool can answer.  For example, asking it to "summarize" the article would launch the summary_tool.

In [7]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the document."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the document."
    ),
)

## Define Router

Llamaindex provides several diffent selectors.  

- The LLM selectors use the LLM to output a JSON that is parsed, and corresponding indexes are queried.
- The Pydantic selects use the OpenAI Function Calling API to produce pydantic selection objects rather than raw JSON.

In this example we are using the LLMSingleSelector. The RouterQueryEngine takes in a selector type as well as a set of tools.

In [8]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    # allows us to view the intermediate steps of the query engine
    verbose=True,
)

In [9]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice is most relevant as it specifically mentions summarization questions related to the document..
The document discusses the importance of bridging the Product Company Gap, which involves moving beyond just product Market fit to create a product that can scale into a successful company. It emphasizes the need to focus on a minimum viable segment, simplify the product installation process, provide instant and ongoing value, and play well in the ecosystem through partnerships. The discussion also covers strategies like offering free trials, creating value through partnerships, and designing a product with a clear pricing model from the beginning. Overall, the document highlights the key steps and considerations for building a successful product and company.


In [10]:
print(len(response.source_nodes))

37


In [11]:
response = query_engine.query("Explain minimum viable segment.")
print(str(response))

Selecting query engine 1: The minimum viable segment refers to a specific context within a document, making choice 2 the most relevant for explaining this concept..
A minimum viable segment is a specific target market that a business focuses on to prove the viability of its product or service. This segment is small enough to allow the business to dominate the market by solving a particular problem effectively. By targeting this minimum viable segment, a company can demonstrate that it can succeed in the market with its initial idea, without needing to make large investments or target a broad audience. The goal is to identify a segment where the business can excel and generate revenue, while also understanding the pain points, budget, product use case, and how to effectively sell to that customer profile.


In [12]:
response = query_engine.query("How do you build a product according to this document?")
print(str(response))

Selecting query engine 1: Useful for retrieving specific context from the document..
To build a product according to this document, you start by designing the product for go-to-market fit, not just product market fit. The first step is to create a minimum viable product (MVP) and ensure that your value proposition is solid before investing further resources. It is crucial to triple check the value proposition to confirm that you are solving a valuable problem that people are willing to pay for. Additionally, you should focus on building a repeatable product that can be deployed effectively. By designing the product with a strong value proposition and ensuring it is viable for the market, you lay the foundation for a successful product that can potentially scale into a company.
